In [1]:
from bloc import *

In [2]:
key = np.random.randint(0, 2**64, dtype=np.uint64)
print(key)

13519796370866853056


In [3]:
m = [np.random.randint(0, 2**64, dtype=np.uint64) for _ in range(16)]
m = int(''.join([bin(m[i])[2:].zfill(64) for i in range(16)]), 2)
print(f"{m:x}")
c = block_encrypt(key=key, message=m, rounds=16)
print(f"{c:x}")
print(f"{block_decrypt(key=key, cipher=c, rounds=16):x}")

11df429e30dc10e8f4d82bdfd3a57fbe6186e165e51c2f2aad0d825859c6f67bd4c901253a621e2c8c7b90fb61f82f1dfb3bc01188a847915d739d6d013076afdd6703b406ef0823d6b8103015232240e7105071e3c408622f96627226ab7e2732724d6b24566fc6705f19f95225f6298753ca4962bea25df35384c9d12983c1
1c097f605debf01e3a080a3789e5b2eccf7e89abedd923b7de4546e3f03f6dd12af9d6f8b577124c5ee550564888a3ccb81cd1c304bae97b710c8f56341446ce876ca418acfcd571ee87fde12f13700362e483cec9561f89902ee7e83e83a94aa9ed088dffe60bb4aaf16f3108855d44df89a87b5a6a89d3d8d8829190b1d0f4
11df429e30dc10e8f4d82bdfd3a57fbe6186e165e51c2f2aad0d825859c6f67bd4c901253a621e2c8c7b90fb61f82f1dfb3bc01188a847915d739d6d013076afdd6703b406ef0823d6b8103015232240e7105071e3c408622f96627226ab7e2732724d6b24566fc6705f19f95225f6298753ca4962bea25df35384c9d12983c1


/Users/neosapien/Development/exam-sg8/sbox.py:24: RuntimeWarning: overflow encountered in scalar multiply
  x *= b
/Users/neosapien/Development/exam-sg8/sbox.py:27: RuntimeWarning: overflow encountered in scalar multiply
  x *= a
/Users/neosapien/Development/exam-sg8/sbox.py:33: RuntimeWarning: overflow encountered in scalar multiply
  x *= a_inv
/Users/neosapien/Development/exam-sg8/sbox.py:36: RuntimeWarning: overflow encountered in scalar multiply
  x *= b_inv
